## Macromolecules : Binding Affinity

LAST REVISED: July 7, 2023

In [1]:
from src.modules.chemicals.chem_data import ChemData
from src.modules.reactions.reaction_dynamics import ReactionDynamics

In [2]:
# Initialize the system
chem = ChemData(names=["A", "B", "C"])


### Explore methods to manage the data structure for macromolecules

In [3]:
chem.add_macromolecules("M1")
chem.get_macromolecules()

['M1']

In [4]:
chem.set_binding_site_affinity("M1", 3, "A", 3.7)
chem.set_binding_site_affinity("M1", 8, "B", 1.1)
chem.set_binding_site_affinity("M1", 15, "A", 9.2)

chem.set_binding_site_affinity("M2", 1, "C", 5.6)

In [5]:
chem.get_binding_sites("M1")

[3, 8, 15]

In [6]:
chem.get_binding_sites_and_ligands("M1")

{3: 'A', 8: 'B', 15: 'A'}

In [7]:
chem.get_binding_sites("M2")

[1]

In [8]:
chem.get_binding_sites_and_ligands("M2")

{1: 'C'}

In [9]:
chem.get_binding_site_affinity(macromolecule="M1", site_number=3)   # A "NamedTuple" gets returned

ChemicalAffinity(chemical='A', affinity=3.7)

In [10]:
aff = chem.get_binding_site_affinity(macromolecule="M1", site_number=8)
aff

ChemicalAffinity(chemical='B', affinity=1.1)

In [11]:
aff.chemical

'B'

In [12]:
aff.affinity

1.1

### Set the initial concentrations of all the chemicals

In [13]:
dynamics = ReactionDynamics(chem_data=chem)
dynamics.set_conc(conc={"A": 20., "B": 0., "C": 10.})
dynamics.describe_state()

SYSTEM STATE at Time t = 0:
3 species:
  Species 0 (A). Conc: 20.0
  Species 1 (B). Conc: 0.0
  Species 2 (C). Conc: 10.0


In [14]:
dynamics.set_macromolecules()      # By default, set counts to 1 for all the registered macromolecules

### Inspect some class attributes

In [15]:
dynamics.macro_system

{'M1': 1, 'M2': 1}

In [16]:
dynamics.macro_system_state

{'M1': {3: ('A', 0.0), 8: ('B', 0.0), 15: ('A', 0.0)}, 'M2': {1: ('C', 0.0)}}

In [17]:
dynamics.describe_state()

SYSTEM STATE at Time t = 0:
3 species:
  Species 0 (A). Conc: 20.0
  Species 1 (B). Conc: 0.0
  Species 2 (C). Conc: 10.0
Macro-molecules present, with their counts:  {'M1': 1, 'M2': 1}
Binding Occupancy fractions at the various binding-sites for each of the macro-molecules:
     M1 | 3: 0.0 (A), 8: 0.0 (B), 15: 0.0 (A)
     M2 | 1: 0.0 (C)
